###### References: 
- https://docs.python.org/3/reference/datamodel.html   
- Fluent Python by Luciano Ramalho. Chapter  1: The Python  Data Model

The Python data model is like a description of Python as a framework.  
It makes use of certain structural guidelines to define its behaviour and usage.  
It formalizes the  interfaces of the building  blocks of the language itself:-
* sequences / collections
* iterators
* Functions and method invocation
* classes
 * Attribute access
 * Operator overloading
* string representation and formatting
* context managers (i.e.  with blocks)


# Special Methods
Also known as dunder methods are meant to be called by the Python interpreter.

More often than not, the special  method call is implicit.

In [1]:
len([2022,1,8])

3

In [2]:
eg = ['the for statement actually  invokes iter(eg)',
      'which in turn may call eg.__iter__() if available']
for item in eg:
    print(item)

the for statement actually  invokes iter(eg)
which in turn may call eg.__iter__() if available


The only special method that is frequently called by user code is `__init__`

To invoke special methods, we use its related build-in function, e.g. len, iter, str, etc.

## A Pythonic Card Deck
The following example demonstrates the power of implementing the two special methods, `__getitem__` and `__len__`
### namedtuple

In [3]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

`namedtuple` is used to build classes of objects that are just bundles of attributes with no custom methods, like a database record.

In [4]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

### A deck as a sequence of cards

In [5]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'Spades Diamonds Clubs Hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits
                                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [6]:
deck = FrenchDeck()
len(deck)

52

Returned by  `__getitem__`:

In [7]:
deck[0]

Card(rank='2', suit='Spades')

In [8]:
deck[-1]

Card(rank='A', suit='Hearts')

Ramdomly select a card:

In [9]:
from random import choice
choice(deck)

Card(rank='5', suit='Spades')

Because the above `__getitem__` delegates to the `[]` operator, the deck automatically supports slicing.

In [10]:
deck[:3]

[Card(rank='2', suit='Spades'),
 Card(rank='3', suit='Spades'),
 Card(rank='4', suit='Spades')]

In [11]:
deck[12::13]

[Card(rank='A', suit='Spades'),
 Card(rank='A', suit='Diamonds'),
 Card(rank='A', suit='Clubs'),
 Card(rank='A', suit='Hearts')]

By implementing the `__getitem__` special method, the above deck is also iterable:

In [12]:
for card in reversed(deck):
    print(card)

Card(rank='A', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank='10', suit='Hearts')
Card(rank='9', suit='Hearts')
Card(rank='8', suit='Hearts')
Card(rank='7', suit='Hearts')
Card(rank='6', suit='Hearts')
Card(rank='5', suit='Hearts')
Card(rank='4', suit='Hearts')
Card(rank='3', suit='Hearts')
Card(rank='2', suit='Hearts')
Card(rank='A', suit='Clubs')
Card(rank='K', suit='Clubs')
Card(rank='Q', suit='Clubs')
Card(rank='J', suit='Clubs')
Card(rank='10', suit='Clubs')
Card(rank='9', suit='Clubs')
Card(rank='8', suit='Clubs')
Card(rank='7', suit='Clubs')
Card(rank='6', suit='Clubs')
Card(rank='5', suit='Clubs')
Card(rank='4', suit='Clubs')
Card(rank='3', suit='Clubs')
Card(rank='2', suit='Clubs')
Card(rank='A', suit='Diamonds')
Card(rank='K', suit='Diamonds')
Card(rank='Q', suit='Diamonds')
Card(rank='J', suit='Diamonds')
Card(rank='10', suit='Diamonds')
Card(rank='9', suit='Diamonds')
Card(rank='8', suit='Diamonds')
Card(r

If a collection has no `__contains__` method the `in` operation does a sequential scan:

In [13]:
Card('Q', 'Hearts') in deck

True

In [14]:
Card('7', 'hearts') in deck

False

## Sorting
There are various ways to rank the cards in a deck.

In [15]:
suit_values = dict(Spades=3, Hearts=2, Diamonds=1, Clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='Clubs')
Card(rank='2', suit='Diamonds')
Card(rank='2', suit='Hearts')
Card(rank='2', suit='Spades')
Card(rank='3', suit='Clubs')
Card(rank='3', suit='Diamonds')
Card(rank='3', suit='Hearts')
Card(rank='3', suit='Spades')
Card(rank='4', suit='Clubs')
Card(rank='4', suit='Diamonds')
Card(rank='4', suit='Hearts')
Card(rank='4', suit='Spades')
Card(rank='5', suit='Clubs')
Card(rank='5', suit='Diamonds')
Card(rank='5', suit='Hearts')
Card(rank='5', suit='Spades')
Card(rank='6', suit='Clubs')
Card(rank='6', suit='Diamonds')
Card(rank='6', suit='Hearts')
Card(rank='6', suit='Spades')
Card(rank='7', suit='Clubs')
Card(rank='7', suit='Diamonds')
Card(rank='7', suit='Hearts')
Card(rank='7', suit='Spades')
Card(rank='8', suit='Clubs')
Card(rank='8', suit='Diamonds')
Card(rank='8', suit='Hearts')
Card(rank='8', suit='Spades')
Card(rank='9', suit='Clubs')
Card(rank='9', suit='Diamonds')
Card(rank='9', suit='Hearts')
Card(rank='9', suit='Spades')
Card(rank='10', suit='Clubs')
Ca

By implementing the special methods `__len__` and `__getitem__`, the above FrenchDeck class behaves like a standards Python sequence, allowing it to benefit from core language features, such as iteration, slicing.

The current implementation however cannot be shuffled, because it is immutable. It can be fixed by adding the `__setitem__` method.

## Vector
### Emulating Numeric Types
Several special methods allow user objects to respond to operators  suce as `+`.

The following implements a class to represent two-dimensional vectors, that is Euclidean vectors like those used in math and physics.

In [16]:
from math import hypot

class Vector:
    def __init__(self,  x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [17]:
v1 = Vector(2,4)
v2 = Vector(2,1)

v1 + v2

Vector(4, 5)

####  `math.hypot(*coordinates)`

Return the Euclidean norm, `sqrt(sum(x**2 for x in coordinates))`. This is the length of the vector from the origin to the point given by the coordinates.

For a two dimensional point `(x, y)`, this is equivalent to computing the hypotenuse of a right triangle using the Pythagorean theorem, `sqrt(x*x + y*y)`.

In [18]:
v = Vector(3,4)
abs(v)

5.0

In [19]:
v * 3

Vector(9, 12)

In [20]:
abs(v*3)

15.0

### String Representation
The string returned by `__repr__` should be unambiguous and, if possible, match the source code necessary to re-create the object being represented.

In contrast, `__str__` called by `str()` should return a string suitable for disply to end users.

### Boolean Value of a Custom Type
The above implementation of `__bool__` is conceptually simple: it returns a `False` if the magnitude of the bector is zero, `True` otherwise.

In [21]:
bool(Vector(0,0))

False

### `len` is not a method
From The Zen of Python, "practicality beats purity", `len(x)` runs very fast because the length is simply read from a field in a C struct as opposed to calculated.

In [22]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
